# microscopy image data readers

> This module contains all code responsible for reading microscopy image files:

In [ ]:
#| default_exp readers/specs

In [ ]:
#| export

from abc import ABC, abstractmethod
from typing import List, Tuple, Optional, Dict, Any
from pathlib import PosixPath, Path

from findmycells.configs import DefaultConfigs, GUIConfigs

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

class ReaderSpecs(ABC):
    
    @property
    @abstractmethod
    def reader_widget_description(self) -> str:
        """
        Description that will be converted into HTML and displayed in the GUI
        right above the constructed widgets. Should therefore explain everything
        relevant for the user to understand the configuration options.
        """
        pass
    
    @property
    @abstractmethod
    def default_configs(self) -> DefaultConfigs:
        """
        Abstract method that requires its subclasses to define the `default_configs`
        as a property of the class. Thus, this will specify all configuration options
        that come with each subclass, while simultaneously also providing default values
        for each option and, moreover, defining what types of values are allowed for each
        option. Check out the implementation of `DefaultConfigs` in the configs module, or
        have a look at how this is implemented in one of the following ReaderSpecs 
        subclasses, like the `MicroscopyReaderSpecs` class.
        """
        pass    
    
    
    @property
    @abstractmethod
    def widget_names(self) -> Dict[str, str]:
        pass
    
    
    @property
    @abstractmethod
    def descriptions(self) -> Dict[str, str]:
        pass
    
    
    @property
    @abstractmethod
    def tooltips(self) -> Optional[Dict[str, str]]:
        return None    
    
    
    def initialize_gui_configs_and_widget(self) -> None:
        gui_configs = GUIConfigs(widget_names = self.widget_names,
                                 descriptions = self.descriptions,
                                 tooltips = self.tooltips)
        gui_configs.construct_widget(strategy_description = 'General processing configurations for this step:',
                                     default_configs = self.default_configs)
        setattr(self, 'gui_configs', gui_configs)
        self.widget = self.gui_configs.strategy_widget
    
    
    def export_current_gui_config_values(self) -> Dict:
        return self.gui_configs.export_current_config_values()    

In [ ]:
#| export

class MicroscopyReaderSpecs:
    
    @property
    def widget_names(self):
        widget_names = {'all_color_channels': 'Checkbox',
                        'specific_color_channel_idxs_range': 'IntRangeSlider',
                        'all_planes': 'Checkbox',
                        'specific_plane_idxs_range': 'IntRangeSlider',
                        'version_idx': 'BoundedIntText',
                        'tile_row_idx': 'BoundedIntText',
                        'tile_col_idx': 'BoundedIntText'}
        return widget_names

    @property
    def descriptions(self):
        descriptions = {'all_color_channels': 'load all color channels (if unchecked, you have to specify the range color channel below)',
                        'specific_color_channel_idxs_range': 'Range of color channel indices to load (only if "all color channels" is unchecked)',
                        'all_planes': 'load all image planes (if unchecked, you have to specify the range of planes below)',
                        'specific_plane_idxs_range': 'Range of plane indices to load (only if "all planes" is unchecked)',
                        'version_idx': '[Optional] Version index to load',
                        'tile_row_idx': '[Optional] Row index of tile to load',
                        'tile_col_idx': '[Optional] Column index of tile to load'}
        return descriptions
    
    @property
    def tooltips(self):
        return {}   
    
    
    @property
    def default_configs(self) -> DefaultConfigs:
        default_values = {'all_color_channels': True,
                          'specific_color_channel_idxs_range': (0, 1),
                          'all_planes': True,
                          'specific_plane_idxs_range': (0, 1),
                          'version_idx': 0,
                          'tile_row_idx': 0,
                          'tile_col_idx': 0}
        valid_types = {'all_color_channels': [bool],
                       'specific_color_channel_idxs_range': [tuple], # a tuple of integers
                       'all_planes': True,
                       'specific_plane_idxs_range': [tuple], # a tuple of integers
                       'version_idx': [int],
                       'tile_row_idx': [int],
                       'tile_col_idx': [int]}
        valid_ranges = {'specific_color_channel_idxs_range': (0, 3, 1), # are more possible? most implementations currently assume single color or RGB
                        'specific_plane_idxs_range': (0, 100, 1), # more should usually not be required?
                        'version_idx': (0, 999, 1), # 999 just to put an upper limit
                        'tile_row_idx': (0, 999, 1), # 999 just to put an upper limit
                        'tile_col_idx': (0, 999, 1)} # 999 just to put an upper limit        
        default_configs = DefaultConfigs(default_values = default_values, valid_types = valid_types, valid_value_ranges = valid_ranges)
        return default_configs

In [ ]:
import ipywidgets as w

In [ ]:
w.IntSlider(description = 'hund', tooltips = 'katze')

IntSlider(value=0, description='hund')

In [ ]:
a[0:10, :, 0:5]

array([[[1, 2, 3]]])

In [ ]:
#| export

class MicroscopyImageReaders(DataReader):
    """
    The read method of MicroscopyImageReaders subclasses has to return a numpy array with the following structure:
    [imaging-planes, rows, columns, color-channels] 
    For instance, an array of a RGB z-stack with 10 image planes of 1024x1024 pixels will have a shape of:
    [10, 1024, 1024, 3]
    To improve re-usability of the same functions for all different kinds of input images, this structure will 
    be used even if there is just a single plane. For instance, the shape of the array of a grayscale 
    2D image with 1024 x 1024 pixels will look like this:
    [1, 1024, 1024, 1]    
    """
    @property
    def default_configs(self) -> DefaultConfigs:
        """
        Commonly only a fraction of the actual microscopy image file needs to analyzed and, therefore,
        loaded. This methods allows the user to define the exact color channel, imaging plane, tile, or
        version of the image data (often the data can be loaded in different resolutions) that needs to be 
        loaded. The corresponding `MicroscopyImageReader` implementations will then ensure the correct 
        selection upon executing the `read()` method.
        """
        return DEFAULT_READER_CONFIGS


    def assert_correct_output_format(self, output: np.ndarray) -> None:
        assert type(output) == np.ndarray, 'The constructed output is not a numpy array!'
        assert len(output.shape) == 4, 'The shape of the to-be-returned array does not match the expected shape!'

In [ ]:
#| export

class CZIReader(MicroscopyImageReaders):
    
    """
    This reader enables loading of images acquired with the ZEN imaging software by Zeiss, using the czifile package.
    Note: the first three dimensions are entirely guessed, it could well be that they reflect different things and 
    not "version_idx", "tile_row_idx", "tile_col_idx"!
    """
    
    @property
    def readable_filetype_extensions(self) -> List[str]:
        return ['.czi']
    
    
    def read(self,
             filepath: Path, # filepath to the microscopy image file
             reader_configs: Dict # the project database
            ) -> np.ndarray: # numpy array with the structure: [imaging-planes, rows, columns, imaging-channel]     
        # To ensure that we don´t lose a dimension if only a single color channel is to be selected:
        if type(reader_configs['color_channel_idx']) == int:
            reader_configs['color_channel_idx'] = slice(reader_configs['color_channel_idx'], reader_configs['color_channel_idx'] + 1)
        read_image_using_configs = czifile.imread(filepath)[reader_configs['version_idx'],
                                                            reader_configs['tile_row_idx'], 
                                                            reader_configs['tile_col_idx'], 
                                                            reader_configs['plane_idx'], 
                                                            :, 
                                                            :, 
                                                            reader_configs['color_channel_idx']]
        return read_image_using_configs

In [ ]:
#| export

class RegularImageFiletypeReader(MicroscopyImageReaders):
    
    """
    This reader enables loading of all regular image filetypes, that scikit-image can read, using the scikit-image.io.imread function.
    Note: So far only single plane images are supported (yet, both single-color & multi-color channel images are supported)!
    """
    
    @property
    def readable_filetype_extensions(self) -> List[str]:
        # ToDo: figure out which formats are possible, probably many many more.. 
        return ['.png', '.tif', '.tiff', '.jpg']
    
    
    def read(self,
             filepath: PosixPath, # filepath to the microscopy image file
             reader_configs: Dict
            ) -> np.ndarray: # numpy array with the structure: [imaging-planes, rows, columns, imaging-channel]
        image_with_correct_format = self._attempt_to_load_image_at_correct_format(filepath = filepath)
        # To ensure that we don´t losse a dimension if only a single color channel is to be selected:
        if type(reader_configs['color_channel_idx']) == int:
            reader_configs['color_channel_idx'] = slice(reader_configs['color_channel_idx'], reader_configs['color_channel_idx'] + 1)
        read_image_using_configs = image_with_correct_format[:, :, :, reader_configs['color_channel_idx']]
        return read_image_using_configs 
    
    
    def _attempt_to_load_image_at_correct_format(self, 
                                                 filepath: PosixPath
                                                ) -> np.ndarray:
        single_plane_image = imread(filepath)
        if len(single_plane_image.shape) == 2: # single color channel
            image_with_correct_format = np.expand_dims(single_plane_image, axis=[0, -1])
        elif len(single_plane_image.shape) == 3: # multiple color channels (at least when assumption of "single plane image" holds)
            image_with_correct_format = np.expand_dims(single_plane_image, axis=[0])
        else:
            raise NotImplementedError('There is something odd with the dimensions of the image you´re attempting to load. '
                                      'It should have either 2 or 3 dimensions, if it is a 2D image with a single color '
                                      'channel, or with multiple color channels, respectively. However, the file you´d like '
                                      f'to load has {len(single_plane_image.shape)} dimensions. For developers: the shape '
                                      f'was: {single_plane_image.shape}.')
        return image_with_correct_format

In [ ]:
#| export

class FromExcelReader(MicroscopyImageReaders):
    
    """
    This reader is actually only a wrapper to the other MicroscopyImageReaders subclasses. It can be used if you stored the filepaths
    to your individual plane images in an excel sheet, for instance if you were using our "prepare my data for findmycells" functions.
    Please be aware that the corresponding datatype has to be loadable with any of the corresponding MicroscopyImageReaders!
    """
    
    @property
    def readable_filetype_extensions(self) -> List[str]:
        # ToDo: figure out which formats are possible, probably many many more.. 
        return ['.xlsx']
        
    
    def read(self,
             filepath: PosixPath, # filepath to the excel sheet that contains the filepaths to the corresponding image files
             reader_configs: Dict
            ) -> np.ndarray: # numpy array with the structure: [imaging-planes, rows, columns, imaging-channel]

        import findmycells.readers as readers
        
        df_single_plane_filepaths = pd.read_excel(filepath)
        single_plane_images = []
        for row_index in range(df_single_plane_filepaths.shape[0]):
            single_plane_image_filepath = Path(df_single_plane_filepaths['plane_filepath'].iloc[row_index])
            file_extension = single_plane_image_filepath.suffix
            image_loader = DataLoader()
            image_reader_class = image_loader.determine_reader(file_extension = file_extension,
                                                               data_reader_module = readers.microscopy_images)
            loaded_image = image_loader.load(data_reader_class = image_reader_class,
                                             filepath = single_plane_image_filepath,
                                             reader_configs = reader_configs)
            single_plane_images.append(loaded_image)
        read_image_using_configs = np.stack(single_plane_images)
        return read_image_using_configs

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()